In [28]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

from collections import defaultdict
%matplotlib inline

In [64]:
df = pd.read_feather('/Users/alexhooker/Downloads/nan_pagesdf.f')
full_df = pd.read_csv('/Users/alexhooker/projects/kaggle/wikits/data/train_1.csv')

In [65]:
df['page_index'] = df['page_index'].astype(int)
df = full_df.loc[df['page_index'].values]
dates = [c for c in df.columns if c != 'Page']
df.drop(dates, axis=1, inplace=True)
# df['Page'] = full_df.loc[df['page_index'].values, 'Page']
df.head(5)

,Page
84677,User_talk:175.44.5.169_www.mediawiki.org_all-a...
81786,Category:Deletion_requests_January_2017_common...
4294,IU_(艺人)_zh.wikipedia.org_all-access_spider
79640,File:Trump_signing_order_January_27_(square_cr...
89819,特別:フィード項目/featured/20160328000000/ja_ja.wikipe...


In [66]:
len(df)*60

71700

In [67]:
key_dict = defaultdict(list)

with open('../data/key_1.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        page_name = '_'.join(row['Page'].split('_')[:-1])
        key_dict[page_name].append(row['Id'])

In [56]:
base_dir = '../data/'
train = pd.read_csv(base_dir+'train_1.csv')

In [57]:
train['median_pred'] = np.nan_to_num(np.round(np.nanmedian(train.drop('Page', axis=1).values[:, -49:], axis=1)).reshape(-1,1))
train['median_pred'].isnull().any()

/Users/alexhooker/projects/informatics/py3env/lib/python3.4/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


False

In [58]:
# target_pages = train['Page'].values
target_pages = df['Page'].values
train = train[train['Page'].isin(target_pages)]

In [68]:
sub_list = []
for page in df['Page']:
    sub_list += [{'Id': pred_id, 'Visits': 0} for pred_id in key_dict[page]]

In [59]:
sub_list = []
for index, row in train[['Page', 'median_pred']].iterrows():
    sub_list += [{'Id': pred_id, 'Visits': row['median_pred']} for pred_id in key_dict[row['Page']]]

In [69]:
sub_list[:60]

[{'Id': 'e33b73546b4f', 'Visits': 0},
 {'Id': 'f344efda51b5', 'Visits': 0},
 {'Id': 'd712f48b1494', 'Visits': 0},
 {'Id': 'e1b08563a382', 'Visits': 0},
 {'Id': 'f041a62ed6c2', 'Visits': 0},
 {'Id': 'e1d0c5ae1e75', 'Visits': 0},
 {'Id': '68567533727a', 'Visits': 0},
 {'Id': 'd9f4208c5dd9', 'Visits': 0},
 {'Id': '7137e5ab1034', 'Visits': 0},
 {'Id': 'f25c2c4e2898', 'Visits': 0},
 {'Id': '7d10d072079f', 'Visits': 0},
 {'Id': '8bef9d92f1a8', 'Visits': 0},
 {'Id': '511cdbafaf31', 'Visits': 0},
 {'Id': '230efcc66537', 'Visits': 0},
 {'Id': '9307631f61e3', 'Visits': 0},
 {'Id': 'c14eedd4268c', 'Visits': 0},
 {'Id': 'f92b448e4829', 'Visits': 0},
 {'Id': '50ec3099335d', 'Visits': 0},
 {'Id': 'aa30d70382bf', 'Visits': 0},
 {'Id': 'e04f7f13c584', 'Visits': 0},
 {'Id': '1f620e81ec65', 'Visits': 0},
 {'Id': 'b6a6d7cbbfa7', 'Visits': 0},
 {'Id': '21340af3ff7b', 'Visits': 0},
 {'Id': '85f4e305624f', 'Visits': 0},
 {'Id': '67e9585c6929', 'Visits': 0},
 {'Id': '3f52912819d1', 'Visits': 0},
 {'Id': '788

In [61]:
np.nanmedian(train.drop('median_pred', axis=1).head(1).values[0,-49:].astype(np.float))

18.0

In [70]:
sub_df = pd.DataFrame(sub_list)
if len(sub_df) == 4988640:
    print('saving selected')
    sub_df.to_feather('median_selected.f')
elif len(sub_df) == 71700:
    print('saving nan')
    sub_df.to_feather('zero_preds.f')
else:
    print('saving all')
    sub_df.to_feather('median_full.f')

saving nan


In [63]:
fd = pd.read_feather('median_selected.f')
len(fd)

4988640

In [72]:
sub_df_full = pd.read_feather('../data/median_full.f')
assert len(sub_df_full) == 8703780

In [75]:
sub_df_full.to_csv('../data/median_full.csv', index=False)